# Video Segmentation with SAM 3

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam3_video_segmentation.ipynb)

This notebook demonstrates how to use SAM 3 for video segmentation and tracking. SAM3 provides:

- **Text prompts**: Segment objects using natural language (e.g., "person", "car")
- **Point prompts**: Add clicks to segment and refine objects
- **Object tracking**: Track segmented objects across all video frames
- **Time series support**: Process GeoTIFF time series with georeferencing


## Installation

SAM3 requires CUDA-capable GPU. Install with:


In [ ]:
# !pip install segment-geospatial[samgeo3]

## Import Libraries


In [ ]:
import os
from samgeo.samgeo3 import SamGeo3Video

## Initialize Video Predictor

The `SamGeo3Video` class provides a simplified API for video segmentation. It automatically uses all available GPUs.


In [ ]:
sam = SamGeo3Video()

## Load a Video

You can load from different sources:
- MP4 video file
- Directory of JPEG frames
- Directory of GeoTIFFs (for remote sensing time series)


In [ ]:
# Using a sample video from SAM3
video_path = os.path.expanduser("~/Documents/Sync/sam3/assets/videos/0001")
sam.set_video(video_path)

## Text-Prompted Segmentation

Use natural language to describe objects. SAM3 finds all instances and tracks them.


In [ ]:
# Segment all people in the video
sam.generate_masks("person")

## Visualize Results


In [ ]:
# Show the first frame with masks
sam.show_frame(0)

In [ ]:
# Show multiple frames in a grid
sam.show_frames(frame_stride=60, ncols=3)

## Remove Objects

Remove specific objects by ID and re-propagate.


In [ ]:
# Remove object 2 and re-propagate
sam.remove_object(2)
sam.propagate()
sam.show_frame(0)

## Point Prompts

Add objects back or refine segmentation using point prompts.


In [ ]:
# Add back object 2 with a positive point click
sam.add_point_prompts(
    points=[[760, 550]],  # [x, y] coordinates
    labels=[1],  # 1=positive, 0=negative
    obj_id=2,
    frame_idx=0,
)
sam.propagate()
sam.show_frame(0)

## Refine with Multiple Points

Use positive and negative points to refine the mask.


In [ ]:
# Refine to segment only the shirt (not pants)
sam.add_point_prompts(
    points=[[740, 450], [760, 630], [840, 640], [760, 550]],
    labels=[1, 0, 0, 1],  # positive, negative, negative, positive
    obj_id=2,
    frame_idx=0,
)
sam.propagate()
sam.show_frames(frame_stride=60, ncols=3)

## Save Results

Save masks as images or create an output video.


In [ ]:
os.makedirs("output", exist_ok=True)

# Save mask images
sam.save_masks("output/masks")

In [ ]:
# Save video with blended masks
sam.save_video("output/segmented.mp4", fps=30)

## Segment Different Objects

Reset and segment different objects with a new text prompt.


In [ ]:
# Segment shoes instead
sam.generate_masks("shoe")
sam.show_frame(0)

## Close Session

Close the session to free GPU resources.


In [ ]:
sam.close()

# To completely shutdown and free all resources:
# sam.shutdown()